Notebook to generate data issues

In [ ]:
# 1 feature_missing => one or some of the csv files are missing a required column.
# 2 values_missing => one or more values in a row are nulls.
# 3 unknown_categorical => expected value "dinner", "lunch", new value: "drinks".
# 4 out_of_range_numerical => wrong value for a feature, e.g:-1 for tips.
# 5 values_types_invalid => string value in a numerical column and vice versa.
# 6 values_outlier => 1000 people for a table.
# 7 values_bad_encoding => having unreadable character in CSV file e.g: à, ù, Ö, Ü.
# 8 csv_format => one or some csv files are seperated by other delimiter e.g: TSV using \t
# 9 header_missing => in a csv file one or more of the label is missing e.g: "tips"

In [9]:
import pandas as pd
data1 = {
    "tips" : [1.01, 1.66, 3.5, 3.31, 3.61],
    "sex" : ["Female", "Male", "Male", "Male", "Female"],
    "smoker" : [False, False, True, True, True],
    "day" : ["Sat", "Sat", "Sun", "Sun", "Mon"],
    "time" : ["Dinner", "Lunch", "Lunch", "Dinner", "Drinks"],
    "size" : [2, 3, 3, 2, 4]
}

data2 = {
    "total_bill" : [16.99, 10.34, -21.01, 23.68, 24.59],
    "tips" : [1.01, None, 3.5, 3.31, 3.61],
    "sex" : ["Female", "Male", "Male", "Mäle", "Female"],
    "smoker" : [False, False, True, True, True],
    "day" : ["Sat", "Sat", "Sun", "Sun", "Mon"],
    "time" : ["Dinner", "Lunch", "Lunch", "Dinner", "Drinks"],
    "size" : ["two", 3, 3, 200, 4]
}

df_error1 = pd.DataFrame(data1)
df_error2to7 = pd.DataFrame(data2)
df_error8_9 = pd.read_csv("../../tests/resources/test_csv_errors_8_9.csv")
print(df_error1)
print()
print(df_error2to7)
print()
print(df_error8_9)

   tips     sex  smoker  day    time  size
0  1.01  Female   False  Sat  Dinner     2
1  1.66    Male   False  Sat   Lunch     3
2  3.50    Male    True  Sun   Lunch     3
3  3.31    Male    True  Sun  Dinner     2
4  3.61  Female    True  Mon  Drinks     4

   total_bill  tips     sex  smoker  day    time size
0       16.99  1.01  Female   False  Sat  Dinner  two
1       10.34   NaN    Male   False  Sat   Lunch    3
2      -21.01  3.50    Male    True  Sun   Lunch    3
3       23.68  3.31    Mäle    True  Sun  Dinner  200
4       24.59  3.61  Female    True  Mon  Drinks    4

  total_bill\ttip\tsex\tsmoker\t\ttime\tsize
0    16.99\t1.01\tFemale\tNo\tSun\tDinner\t2
1      10.34\t1.66\tMale\tNo\tSun\tDinner\t3
2       21.01\t3.5\tMale\tNo\tSun\tDinner\t3
3      23.68\t3.31\tMale\tNo\tSun\tDinner\t2
4    24.59\t3.61\tFemale\tNo\tSun\tDinner\t4
